In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [0]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [0]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [0]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [0]:
users['month']=pd.DatetimeIndex(users['created_date']).month
users['year']=pd.DatetimeIndex(users['created_date']).year
users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [0]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [0]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [0]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [0]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [0]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [131]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

ea_merchant_country,ABW,AGB,AGO,AIA,ALB,AND,APT,ARE,ARG,ARM,ATG,AUS,AUT,AZE,BEL,BEN,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BWA,CAN,CHE,CHL,CHN,CIV,CMR,COK,COL,CPV,CRI,...,RWA,SAU,SEN,SGB,SGP,SLV,SMR,SRB,SUR,SVK,SVN,SWE,SWZ,SXM,SYC,TCA,TCD,THA,TKM,TTO,TUN,TUR,TWN,TZA,UGA,UKR,URY,USA,UZB,VAT,VCT,VEN,VGB,VIR,VNM,VUT,ZAF,ZAR,ZMB,ZWE
transactions_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ATM,0,4,0,0,39,25,0,303,234,35,8,535,415,18,381,5,8,4,422,5,11,60,22,24,8,153,282,47,2,36,342,443,91,267,44,72,3,524,10,132,...,0,71,57,0,326,0,2,102,2,240,146,168,0,8,11,0,6,1428,0,5,196,733,88,22,96,256,12,1674,8,0,1,0,0,0,873,1,208,2,4,0
CARD_PAYMENT,2,0,2,5,35,436,2,4370,1775,48,100,16782,5715,30,6276,4,0,3,1774,104,91,112,438,16,101,120,3307,169,6,26,8320,11623,1069,518,48,0,19,1365,45,638,...,1,185,75,1,3881,20,4,452,2,3394,2280,11572,3,16,48,9,3,2835,1,62,169,4139,333,43,68,1892,204,55139,8,71,3,140,16,5,1826,11,4092,2,17,37
CARD_REFUND,0,0,0,0,0,0,0,14,67,0,0,120,8,1,25,0,2,0,3,0,0,0,6,0,0,1,19,0,0,0,59,55,2,25,0,0,0,1,1,3,...,0,1,0,0,25,0,0,0,0,7,1,58,0,0,0,0,0,18,0,0,3,5,1,0,0,9,65,502,0,0,0,0,0,0,12,0,9,0,0,0


In [132]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

ea_cardholderpresence,FALSE,TRUE,UNKNOWN
transactions_type,,,
ATM,61512,0,2
CARD_PAYMENT,847184,150467,1761
CARD_REFUND,29,0,2


In [0]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [0]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [0]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [0]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [0]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [0]:
lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [0]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [0]:
trans_sorted=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [0]:
lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [0]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [0]:
users['transactions_frequency']=users['total_amount_of_transactions']/(users['months_from_first_transaction']-users['months_from_last_transaction'])

#Notifications
________________________________________________________


In [0]:
notif_sent=notif[notif['status']=='SENT']


In [0]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=[notif_sent['channel'],notif_sent['reason']])
amount_notif_sent.columns=['_'.join(col).lower() for col in amount_notif_sent.columns.values]

In [0]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [0]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [0]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [0]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [0]:
#users[users['user_id']=='user_12544']

In [0]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [0]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [0]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [0]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [0]:
#dropping columns we don't need
data=users.drop(columns=['user_id','birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','year_x','num_referrals','num_successful_referrals','month_x','created_date_y','month_y','year_y','created_date','month','year'])

In [0]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [0]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [0]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [0]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [0]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
#datatest.to_csv('data_for_testing.csv',index=False, sep=';')
#data.to_csv('data_from_users.csv',index=False, sep=';')
#users.to_csv('users_featured.csv',index=False, sep=';')

In [0]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values